# **Quantum neural network for classification**
---
<img src="http://www.doc.ic.ac.uk/~afd/images/logo_imperial_college_london.png" align = "left" width=200>
 <br><br><br><br>
 
- Copyright (c) Antoine Jacquier, 2024. All rights reserved

- Author: Jack Jacquier <a.jacquier@imperial.ac.uk>

- Platform: Tested on Windows 10 with Python 3.9

We work here on a $\{0,1\}$ binary classification problem. We therefore only use 1 qubit.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import *
from qiskit.quantum_info import state_fidelity
from sklearn.metrics import log_loss
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_moons
import matplotlib.pylab as plt

# Generating (noisy) data

In [ ]:
def generate_data(n_samples, noise, threshold):
    samplePoints = np.random.rand(n_samples, 2)
    labels = list(map(lambda xx: 1*(xx[0]**2 + xx[1]**3 >= threshold), samplePoints))
    ## {0,1} labels depending on whether a point in samplePoints is below or above the diagonal
    
    ################################################################
    ####### create some noise by swapping some of the labels #######
    ################################################################
    
    if noise[0] == 1: ## Swaps some labels randomly (in proportion "noise[1]")
        n = int(np.floor(noise[1]*n_samples))
        noise_sample = np.random.randint(0,n_samples, n)

        for i in noise_sample:
            labels[i] = int((1-labels[i])**2)
    
    else: ## Swaps some labels in the corners
        for (i,s) in enumerate(samplePoints):
            if ((s[0] < noise[1]) and (s[1] > 1.- noise[1])):
                labels[i] = 1
            if ((s[0] >  1.-noise[1]) and (s[1] < noise[1])):
                labels[i] = 0
        
    labels = np.reshape(labels, (len(labels), ))
    
    return samplePoints, labels

In [ ]:
def createData_test_train(generated_data, labels):
    """
    Creates test and training data sets with their labels
    """
    
    x_train, x_test, y_train, y_test = train_test_split(generated_data, labels, stratify=labels, test_size=None, random_state=None)
    
    scale = MinMaxScaler(feature_range=(-1,1)).fit(x_train)
    x_train = scale.transform(x_train)
    x_test = scale.transform(x_test)
    
    return x_train, y_train, x_test, y_test

## Examples

Run only one of the examples below

In [ ]:
n_samples = 200

#### Example 1

Generate random numbers in the unit square and assign them {0,1} labels if they are above or below the diagonal.
The noise swaps some labels randomly.

In [ ]:
threshold = 0.7
noise = [1, 0.0] 

generated_data, labels = generate_data(n_samples, noise, threshold)

#### Example 2

Generate random numbers in the unit square and assign them {0,1} labels if they are above or below the diagonal.
The noise swaps labels in the corners.

In [ ]:
threshold = 0.7
noise = [0, 0.2] ## Noise to swap some label in the corners
generated_data, labels = generate_data(n_samples, noise, threshold)

## Visualise data

Create the training and test sets and plot the train set

In [ ]:
### Split training and test sets
x_train, y_train, x_test, y_test = createData_test_train(generated_data, labels)

In [ ]:
plt.scatter(x_train[:,0][y_train==0], x_train[:,1][y_train==0], s=20, facecolors='none', edgecolors='r')
plt.scatter(x_train[:,0][y_train==1], x_train[:,1][y_train==1], marker='+', color='blue')
plt.title("Original labelled data (with some noise)")
plt.show()

# Quantum circuit for classification

We construct a simple one-qubit classifier with the generic U3 quantum gate, detailed at https://qiskit.org/documentation/stubs/qiskit.circuit.library.U3Gate.html
Note that it is now called `circuit.u`

In [ ]:
class QNN():
    def __init__(self, n_layers, x_train, y_train, x_test, y_test, learning_rate):
        self.n_layers = n_layers
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.state_labels = np.zeros((2,2),dtype=np.complex128)
        self.state_labels[0,0] = 1.
        self.state_labels[-1,-1] = 1.
        self.gradient_shift = .5*np.pi
        self.learning_rate = learning_rate
        self.backend_sim = Aer.get_backend('statevector_simulator')
        
        self.weights = 2.*np.pi*np.random.rand(self.n_layers, 3) ## random initialisation of the weights
        
        
    def circuit_for_plotting(self, x):
        """
        Builds the circuit -- simply for plotting purposes
        x: sample
        """

        self.qc = QuantumCircuit(1)
        self.qc.u(x[0], x[1], 0.,0)

        ### Details of the U gate available here:
        ### https://docs.quantum.ibm.com/api/qiskit/0.24/qiskit.circuit.library.UGate

        for l in range(self.n_layers):
            self.qc.u(self.weights[l][0], self.weights[l][1], self.weights[l][2],0)
    
    def circuit(self, x):

        self.qc = QuantumCircuit(1)
        
        self.qc.u(x[0], x[1], 0., 0) ## Encoding the data
            
        for l in range(self.n_layers):
            self.qc.u(self.weights[l][0], self.weights[l][1], self.weights[l][2],0)


        job_sim = execute(self.qc, self.backend_sim)

        result_sim = job_sim.result()

        state_vector = result_sim.get_statevector(self.qc)

        self.fidelity_1 = state_fidelity(state_vector,self.state_labels[0])
        self.fidelity_2 = state_fidelity(state_vector,self.state_labels[1])
        

    def cost(self, x, y):
        """
        Cross entropy cost from the computed fidelities and the corresponding labels
        """
        
        value = []
        for xTemp in x:
            self.circuit(xTemp)
            value.append([self.fidelity_1, self.fidelity_2])

        return log_loss(y, value)
    

    def gradient(self, x, y):
        """
        Computes the gradients
        x: sample
        y: corresponding label
        """

        #####   We follow https://arxiv.org/pdf/1811.11184.pdf for the computation of the gradient ##### 

        g = np.zeros(self.weights.shape)

        for l in range(self.n_layers):
            for i in range(len(self.weights[l])):

                original_value = self.weights[l][i]

                self.weights[l][i] = original_value + self.gradient_shift
                F_plus = self.cost(x, y)

                self.weights[l][i] = original_value - self.gradient_shift
                F_minus = self.cost(x, y)

                self.weights[l][i] = original_value

                g[l][i] = .5 * (F_plus - F_minus)
        return g
    
    
    def optimize(self, x, y):
        """
        Stochastic gradient algorithm
        """
        self.weights = self.weights - self.learning_rate * self.gradient(x, y)
    
    
    def predict(self, x):
        """
        Prediction function, returns in [0,1]
        """

        self.circuit(x)

        if self.fidelity_1 > self.fidelity_2:
            return abs(1.-self.fidelity_1)
        else:
            return self.fidelity_2

## Creating and viewing the circuits

`n_layers` is the number of unitary gates,  representing the complexity of the quantum neural network. 

There are three parameters (to optimise) per gate, so the larger the number of layers, the harder it is to train.

The parameter `learning_rate` is needed for the stochastic gradient algorithm.

In [ ]:
n_layers = 5
learning_rate = 0.01

#### Creates an instance of the circuit

In [ ]:
qnn = QNN(n_layers, x_train, y_train, x_test, y_test, learning_rate)

Visualisation of the quantum circuit

In [ ]:
qnn.circuit_for_plotting(x_test[0])
print(x_test[0])
qnn.qc.draw(output='mpl')

Note: the first gate encodes the data (x[0],x[1]]), for each point x in the data set (recall that x represents the coordinates) in a quantum unitary gate.

We could alternatively encode the two-dimensional vector x in a one-qubit quantum state after normalisation (probabilities summing up to one).

## Running the learning algorithm

We optimise the system using classical stochastic gradient algorithm with mini-batches

In [ ]:
nb_epochs = 500
batch_size = 50

In [ ]:
current_loss = np.inf
total_losses = []
for epoch in range(nb_epochs):
    
    # mini-batches
    batch_indices = np.random.randint(0,len(x_train), batch_size)
    x_batch, y_batch = x_train[batch_indices], y_train[batch_indices]
    
    qnn.optimize(x_batch, y_batch)
    
    res = qnn.cost(x_test, y_test)
    
    total_losses.append(res)

    if res < current_loss:
        current_loss = res
        var = qnn.weights
        
    print("Epoch: {:2d} | Testing loss: {:4f}".format(epoch+1, res))

### Plot the evolution of the loss function

In [ ]:
plt.xlabel('Number of epochs')
plt.ylabel('Loss')
plt.plot(range(len(total_losses)),total_losses)
plt.title("Loss function")
plt.show()

## Visualising the power of the quantum classifier

We generate many points $n_{points}^2$ on the square and colour-highlight their predicted labels given the optimised quantum cicuit

In [ ]:
axMin, axMax = -1.1, 1.1
n_points = 20
xx, yy = np.meshgrid(np.linspace(axMin, axMax, n_points), np.linspace(axMin, axMax, n_points))

xx_grid = [np.array([x, y]) for x, y in zip(xx.flatten(), yy.flatten())]

predictions_grid = [qnn.predict(x) for x in xx_grid]

In [ ]:
Z = np.reshape(np.array(predictions_grid).round(), xx.shape)

cm = plt.cm.RdBu
cnt = plt.contourf(xx, yy, Z, levels=np.arange(0., 1.1, 0.1), cmap=cm, alpha=.2)

plt.scatter(x_train[:, 0][y_train==0], x_train[:, 1][y_train==0], c='r', marker='+')
plt.scatter(x_train[:, 0][y_train==1], x_train[:, 1][y_train==1], c='b', marker='+')

plt.scatter(np.array(x_test[:, 0][y_test==0]), np.array(x_test[:, 1][y_test==0]), s=20, facecolors='none', edgecolors='r')
plt.scatter(x_test[:, 0][y_test==1], x_test[:, 1][y_test==1], s=80, facecolors='none', edgecolors='b')

plt.show()

The crosses correspond to the training set (over which the quantum network is optimised), while the circles correspond to the test set. 

The shaded regions are the predicted labels for a large number of points in the square.